In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/UCSD"
%cd "/content/drive/My Drive/UCSD"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UCSD


In [ ]:
!pip install keras_layer_normalization

In [ ]:
from os import listdir
import fnmatch
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
np.set_printoptions(threshold=np.inf)
import keras
from keras.layers import Conv2DTranspose,BatchNormalization, Conv2D,MaxPooling2D, UpSampling2D, Input,UpSampling3D,MaxPooling3D
from keras.models import Model
from keras.layers import Conv2DTranspose, ConvLSTM2D,TimeDistributed, Conv2D
from keras.layers import Conv3D,Conv3DTranspose
from keras.layers.normalization import BatchNormalization
# from keras_layer_normalization import LayerNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)
import matplotlib.pyplot as plt
from numpy import moveaxis
import keras.backend as K
# from persistence1d import RunPersistence
from scipy.signal import argrelextrema
from keras.models import load_model
import cv2
from numpy import savez_compressed
import shutil
from numpy import load
def _get_available_gpus():
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]
tfback._get_available_gpus = _get_available_gpus
from keras.utils.vis_utils import plot_model
from skimage.measure import compare_ssim
from skimage.metrics import structural_similarity as ssim

Using TensorFlow backend.


tf.__version__ is 2.2.0
tf.keras.__version__ is: 2.3.0-tf


In [ ]:
folder_name = str(input("Which folder do you want to test your results on?\n"))

Which folder do you want to test your results on?
Test019


In [ ]:
train_path = path + '/UCSD_Anomaly_Dataset.v1p2'
test_path = train_path + '/UCSDped1/Test/'+folder_name

# **2D Convolutional Autoencoder with MSE loss**

In [ ]:
def get_clips_by_stride(stride, frames_list, sequence_size):
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

In [ ]:
training_set = []
for f in sorted(listdir(Config.DATASET_PATH)):
  if fnmatch.fnmatch(f,"*ped*"):
    for g in listdir(join(Config.DATASET_PATH,f)):
      if fnmatch.fnmatch(g,"Train"):
        for c in listdir(join(join(Config.DATASET_PATH,f), g)):
          if fnmatch.fnmatch(c,"Train*"):
            all_frames = []
            for j in listdir(join(join(join(Config.DATASET_PATH,f), g),c)):
              if str(j)[-3:] == "tif":
                img = Image.open(join(join(join(join(Config.DATASET_PATH,f), g),c), j)).resize((256, 256))
                img = np.array(img, dtype=np.float16) / 255.0
                all_frames.append(img)
            # get the augmented images from the list of images after applying data augmentation
            for stride in range(1, 4):
              training_set.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=5))

In [ ]:
input_frm = Input(shape=(5, 256, 256))  

#Encoder
x = Conv2D(512, (11, 11),strides=4,activation='relu', padding='same',
           kernel_initializer=keras.initializers.glorot_normal(),
           data_format='channels_first')(input_frm)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)



x = Conv2D(256, (5, 5), activation='relu', padding='same',
           data_format='channels_first',
           kernel_initializer=keras.initializers.glorot_normal())(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)



encoded = Conv2D(128, (3, 3), activation='relu', padding='same',
                 data_format='channels_first',
                 kernel_initializer=keras.initializers.glorot_normal())(x)

#Decoder
x = Conv2DTranspose(128, (3, 3), padding="same",
                    kernel_initializer=keras.initializers.glorot_normal(),
                    activation='relu',
                    data_format='channels_first')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

x = Conv2DTranspose(256, (3, 3), padding="same",
                    kernel_initializer=keras.initializers.glorot_normal(),
                    activation='relu',
                    data_format='channels_first')(x)
x = BatchNormalization()(x)                    
x = UpSampling2D((2, 2))(x)

x = Conv2DTranspose(512, (5, 5), padding="same",
                    kernel_initializer=keras.initializers.glorot_normal(),
                    activation='relu',
                    data_format='channels_first')(x)

decoded = Conv2DTranspose(5, (11, 11),strides=4, padding="same",
                          kernel_initializer=keras.initializers.glorot_normal(),
                          activation='relu',
                          data_format='channels_first')(x)


conv2d_mse_autoencoder = Model(input_frm, decoded)
conv2d_mse_autoencoder.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.0001), loss='mse')                          

In [ ]:
training_set = np.array(training_set)
training_set = training_set.reshape(-1,5,256,256)

In [ ]:
history = conv2d_mse_autoencoder.fit(training_set,training_set,
                                  epochs=50,
                                  batch_size=32,
                                  shuffle=True
                                )

In [ ]:
sz = 200
test = np.zeros(shape=(sz, 256, 256))
cnt = 0
for f in sorted(listdir(test_path)):
    if str(join(test_path, f))[-3:] == "tif":
      img = Image.open(join(test_path,f)).resize((256, 256))
      img = np.array(img, dtype=np.float16) / 256.0
      test[cnt, :, :] = img
      cnt = cnt + 1

In [ ]:
sz = test.shape[0] - 5
sequences = np.zeros((sz+1, 5, 256, 256))
# apply the sliding window technique to get the sequences
for i in range(0, sz+1):
    clip = np.zeros((5, 256, 256))
    for j in range(0, 5):
        clip[j] = test[i + j, :, :]
    sequences[i]=clip
print("sequences shape:",sequences.shape)

In [ ]:
reconstructed_sequences = conv2d_mse_autoencoder.predict(sequences)

In [ ]:
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz+1)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

In [ ]:
plt.plot(sr)
plt.title(folder_name)
plt.xticks(np.arange(0, sz+1,10))
plt.ylabel('regularity score Sr(t)')
plt.xlabel('frame t')
plt.grid(b=1,axis='both')
plt.rcParams["figure.figsize"] = [16,9]
plt.show()

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
fps = 10
file_name = 'conv2d_mse_'+folder_name+'.mp4'
video_filename = file_name
out = cv2.VideoWriter(video_filename, fourcc, fps, (256, 256))


THRESH = 60
THRESH_AREA = 50

for i in range(len(sequences)):
  input_frame,output_frame = sequences[i][0].copy(),reconstructed_sequences[i][0].copy()
  (score, diff) = ssim(input_frame,output_frame, full=True)
  diff = (diff * 255).astype(np.uint8)
  ret,thresh = cv2.threshold(diff,THRESH,255,cv2.THRESH_BINARY_INV)
  contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  contours = contours[0] if len(contours) == 2 else contours[1]

  for c in contours:
    area = cv2.contourArea(c)
    if area > THRESH_AREA:
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(input_frame, (x, y), (x + w, y + h),1)
  gray = cv2.normalize(input_frame, None, 255, 0,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)
  gray_3c = cv2.merge([gray, gray,gray])
  out.write(gray_3c)

out.release()

# **2D Convolutional Autoencoder with SSIM loss**

In [ ]:
def get_clips_by_stride(stride, frames_list, sequence_size):
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(256, 256,sequence_size))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[:, :,cnt] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

In [ ]:
training_set = []
for f in sorted(listdir(train_path)):
  if fnmatch.fnmatch(f,"*ped*"):
    for g in listdir(join(train_path,f)):
      if fnmatch.fnmatch(g,"Train"):
        for c in listdir(join(join(train_path,f), g)):
          if fnmatch.fnmatch(c,"Train*"):
            all_frames = []
            for j in listdir(join(join(join(train_path,f), g),c)):
              if str(j)[-3:] == "tif":
                img = Image.open(join(join(join(join(train_path,f), g),c), j)).resize((256, 256))
                img = np.array(img, dtype=np.float16) / 255.0
                all_frames.append(img)
            # get the augmented images from the list of images after applying data augmentation
            for stride in range(1, 4):
              training_set.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=5))

In [ ]:
input_frm = Input(shape=(256, 256,5))  

#Encoder
x = Conv2D(512, (11, 11),strides=4,activation='relu', padding='same',kernel_initializer=keras.initializers.glorot_normal())(input_frm)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(256, (5, 5), activation='relu', padding='same',kernel_initializer=keras.initializers.glorot_normal())(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

encoded = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer=keras.initializers.glorot_normal())(x)

#Decoder
x = Conv2DTranspose(128, (3, 3), padding="same",kernel_initializer=keras.initializers.glorot_normal(), activation='relu')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

x = Conv2DTranspose(256, (3, 3), padding="same",kernel_initializer=keras.initializers.glorot_normal(),activation='relu')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)

x = Conv2DTranspose(512, (5, 5), padding="same",kernel_initializer=keras.initializers.glorot_normal(),activation='relu')(x)
x = BatchNormalization()(x)

output_frm = Conv2DTranspose(5, (11, 11),strides=4, padding="same",kernel_initializer=keras.initializers.glorot_normal(),activation='relu')(x)

In [ ]:
def ssim(input_frm, decoded):
    ssim_loss = tf.reduce_mean(tf.image.ssim(output_frm, input_frm, 1.0),axis=-1)
    return 1- ssim_loss

In [ ]:
conv2d_ssim_autoencoder = Model(input_frm,output_frm)
conv2d_ssim_autoencoder.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.001), loss=ssim)

In [ ]:
history = conv2d_ssim_autoencoder.fit(training_set,training_set,
                          epochs=50,
                          batch_size=32,
                          shuffle=True,
                          )

In [ ]:
sz = 200
test = np.zeros(shape=(256, 256,sz))
cnt = 0
for f in sorted(listdir(test_path)):
    if str(join(test_path, f))[-3:] == "tif":
        img = Image.open(join(test_path, f)).resize((256, 256))
        img = np.array(img, dtype=np.float32) / 256.0
        test[:, :, cnt] = img
        cnt = cnt + 1

In [ ]:
sz = test.shape[-1] - 5
sequences = np.zeros((sz+1, 256,256,5))
# apply the sliding window technique to get the sequences
for i in range(0, sz+1):
    clip = np.zeros((256,256,5))
    clip = moveaxis(clip, 2, 0)
    for j in range(0, 5):
        clip[j] = test[:, :,i + j]
    clip = moveaxis(clip, 0, 2)    
    sequences[i] = clip
print("sequences shape:",sequences.shape)

In [ ]:
reconstructed_sequences = autoencoder.predict(sequences)

In [ ]:
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz+1)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

In [ ]:
plt.plot(sr)
plt.title(folder_name)
plt.xticks(np.arange(0, sz+1,10))
plt.ylabel('regularity score Sr(t)')
plt.xlabel('frame t')
plt.grid(b=1,axis='both')
plt.rcParams["figure.figsize"] = [16,9]
plt.show()

In [ ]:
reconstructed_sequences = moveaxis(reconstructed_sequences, 3, 1)
sequences = moveaxis(sequences, 3, 1)

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
fps = 10
file_name = 'conv2d_ssim_'+folder_name+'.mp4'
video_filename = file_name
out = cv2.VideoWriter(video_filename, fourcc, fps, (256, 256))


THRESH = 80
# THRESH_AREA = 50

for i in range(len(sequences)):
  input_frame,output_frame = sequences[i][0].copy(),reconstructed_sequences[i][0].copy()
  (score, diff) = ssim(input_frame,output_frame, full=True)
  diff = (diff * 255).astype(np.uint8)
  ret,thresh = cv2.threshold(diff,THRESH,255,cv2.THRESH_BINARY_INV)
  # contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  # contours = contours[0] if len(contours) == 2 else contours[1]

  # for c in contours:
  #   area = cv2.contourArea(c)
  #   if area > THRESH_AREA:
  #     x,y,w,h = cv2.boundingRect(c)
  #     cv2.rectangle(input_frame, (x, y), (x + w, y + h),1)
  gray = cv2.normalize(thresh, None, 255, 0,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)
  gray_3c = cv2.merge([gray, gray,gray])
  out.write(gray_3c)

out.release()

## **3D Convolutional AE + 2D ConvLSTM & MSE loss**

In [ ]:
def get_clips_by_stride(stride, frames_list, sequence_size):
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(227, 227,sequence_size,1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[:,:,cnt,0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

In [ ]:
training_set = []
i=0
for f in sorted(listdir(train_path)):
  if fnmatch.fnmatch(f,"*ped*"):
    for g in listdir(join(train_path,f)):
      if fnmatch.fnmatch(g,"Train"):
        for c in listdir(join(join(train_path,f), g)):
          if fnmatch.fnmatch(c,"Train*"):
            all_frames = []
            for j in listdir(join(join(join(train_path,f), g),c)):
              if str(j)[-3:] == "tif":
                img = Image.open(join(join(join(join(train_path,f), g),c), j)).resize((227, 227))
                img = np.array(img, dtype=np.float16) / 255.0
                all_frames.append(img)                
            # get the augmented images from the list of images after applying data augmentation
            for stride in range(1, 4):
              training_set.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=5))

In [ ]:
input_frm = Input(shape=(227,227,5,1))
l1 = Conv3D(128, (11, 11,1), strides=(4,4,1), padding="valid",activation='relu',kernel_initializer=keras.initializers.glorot_normal())(input_frm)
l2 = LayerNormalization()(l1)

spatial_encoder = Conv3D(64, (5, 5,1), strides=(2,2,1), padding="valid",activation='relu',kernel_initializer=keras.initializers.glorot_normal())(l2)
spatial_encoder = LayerNormalization()(spatial_encoder)

# # # # #
temporal_encoder = ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True,dropout=0.4,recurrent_dropout = 0.3,kernel_initializer=keras.initializers.glorot_normal())(spatial_encoder)
l5 = LayerNormalization()(temporal_encoder)

l6 = ConvLSTM2D(32, (3, 3), padding="same",return_sequences=True,kernel_initializer=keras.initializers.glorot_normal(),dropout=0.3)(l5)
l7 = LayerNormalization()(l6)

temporal_decoder = ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True,kernel_initializer=keras.initializers.glorot_normal(),dropout=0.2)(l7)
temporal_decoder = LayerNormalization()(temporal_decoder)
# # # # #

l11 = Conv3DTranspose(128, (5, 5,1), strides=(2,2,1), padding="valid",activation='relu',kernel_initializer=keras.initializers.glorot_normal())(temporal_decoder)
l12 = LayerNormalization()(l11)

output_frm = Conv3DTranspose(1, (11, 11,1),strides=(4,4,1), activation="sigmoid", padding="valid",kernel_initializer=keras.initializers.glorot_normal())(l12)

In [ ]:
stae_autoencoder = Model(input_frm,output_frm)
stae_autoencoder.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.001), loss='mse')

In [ ]:
history = stae_autoencoder.fit(training_set,training_set,
                epochs=25,
                batch_size=32,
                shuffle=True
                )

In [ ]:
sz = 200
test = np.zeros(shape=(sz,227, 227,1))
cnt = 0
for f in sorted(listdir(test_path)):
    if str(join(test_path, f))[-3:] == "tif":
        img = Image.open(join(test_path, f)).resize((227, 227))
        img = np.array(img, dtype=np.float16) / 255.0
        test[cnt,:, :,0] = img
        cnt = cnt + 1

In [ ]:
sz = test.shape[0]-5
sequences = np.zeros((sz+1, 227,227,5,1))
for i in range(0, sz+1):
    clip = np.zeros((227, 227,5,1))
    clip = moveaxis(clip, 2, 0)
    for j in range(0, 5):
        clip[j] = test[i+j,:, :,:]
    clip = moveaxis(clip, 0, 2)    
    sequences[i] = clip     

In [ ]:
reconstructed_sequences = autoencoder.predict(sequences)

In [ ]:
sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
sr = 1.0 - sa

In [ ]:
plt.plot(sr)
plt.title(folder_name)
plt.xticks(np.arange(0, sz+1,10))
plt.ylabel('regularity score Sr(t)')
plt.xlabel('frame t')
plt.grid(b=1,axis='both')
plt.rcParams["figure.figsize"] = [16,9]
plt.show()

In [ ]:
reconstructed_sequences = moveaxis(reconstructed_sequences, 3, 1)
reconstructed_sequences = moveaxis(reconstructed_sequences, 4, 2)
sequences = moveaxis(sequences, 3, 1)
sequences = moveaxis(sequences, 4, 2)

In [ ]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
fps = 10
file_name = 'conv2d_mse_'+folder_name+'.mp4'
video_filename = file_name
out = cv2.VideoWriter(video_filename, fourcc, fps, (256, 256))


THRESH = 100
THRESH_AREA = 30

for i in range(len(sequences)):
  input_frame,output_frame = sequences[i][0][0].copy(),reconstructed_sequences[i][0][0].copy()
  (score, diff) = ssim(input_frame,output_frame, full=True)
  diff = (diff * 255).astype(np.uint8)
  ret,thresh = cv2.threshold(diff,THRESH,255,cv2.THRESH_BINARY_INV)
  contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  contours = contours[0] if len(contours) == 2 else contours[1]

  for c in contours:
    area = cv2.contourArea(c)
    if area > THRESH_AREA:
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(input_frame, (x, y), (x + w, y + h),1)
  gray = cv2.normalize(input_frame, None, 255, 0,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)
  gray_3c = cv2.merge([gray, gray,gray])
  out.write(gray_3c)

out.release()